### Shit I did in order
------
#### Process
1. Cleaned raw data
2. Geocoded Cleaned Data
3. Combined two data (cleaned and geocoded) batches into one csv file
4. Download Shapefiles from: [github shapefiles](https://github.com/altcoder/philippines-psgc-shapefiles.git)
5. Convert Lat/Long into Brangay using the shapefiles
    - Program: `assign_barangay.py`
    - Output: `barangay_sightings.csv`
6. Aggregate daily counts per barangay
    - Program: `aggregate_daily_counts.py`
    - Output: `daily_counts.csv`
7. Generate complete daily time series grid
    - Program: `build_time_grid.py`
    - Output: `daily_time_grid.csv`
8. Poisson Distriubtion
    - We used a Poisson model because snake sightings are rare, random events that occur over time, which fits how Poisson processes describe count data. The redistribution smooths extreme sparsity without altering real sightings, preserving actual events while improving statistical stability for modeling.
        - Program: `poisson_redistribution.py`
        - Output: `redistributed counts`
    - Additionally:
        - No actual recorded snake encounters were modified.
        - Only completely empty days were slightly smoothed.
        - No artificial inflation of peak events. (Original Max: 12) (Poisson Max: 12)
        - Small increase in average intensity to reduce extreme sparsity while maintaining realism.
9. Collect ENV data
    - Very long tedious process collecting environmental data from google earth api engine
    - Also merged the different env files since I had to export by month, otherwise it hits the memory limit
        - Program: `merge_csv.py`
10. Clean environmental data
    - Must resort the environmental data and ensure the `adm4_psgc` is `int` instead of `float` in both redistributed count and environmental data.
        - Program: `clean_env_count.py`
        - Output: `environmental_data_clean.csv` and `redistributed_counts_clean.csv`
11. Building Model Dataset
    - combining the snake count and environmental data into one complete table, so each barangay and date has both the sighting count and environmental conditions of the same day. Makes data usable for the model, since it has all the relevant information in one place to learn patterns properly.
        - Program: `build_model_dataset.py`
        - Output: `model_dataset_raw.csv`
12. Interpolation
    - Because there are days with missing env data, and neural networks can't work with missing values, we did interpolation to simply estimate the missing values using nearby days, so the time series stay continous without intervening new events or changing the overall ecological pattern.
        - Program: `interpolation.py`
        - Output: `model_dataset_cleaned.csv`
13. Normalize
    - normalized the environmental data so that all variables are on the same scale, preventing larger numbers (like temperature) from overpowering smaller ones (like soil moisture) during model training
        - Program: `normalize_features.py`
        - Output: `model_dataset.csv`
14. Build Sequence
    - converted the daily data into small 14-day “chunks” so the model can learn how environmental conditions over time influence snake encounters. Each chunk teaches the model how the past two weeks relate to the next day’s snake activity.
        - Program: `build_sequence.py`
        - Output: `data/sequence`

------
#### File references:

- `clean_csv.py` : Cleans and standardizes location entries by removing informal or irrelevant scraped terms (e.g., “po”, “ty”, “tia”).
- `geocode_csv.py` : Converts the *location* column into geographic coordinates, generating corresponding *lat* and *long* columns.
- `merge_csv.py` : Merges multiple CSV datasets into a single file (e.g., combining batch 1 and batch 2).
- `assign_barangay.py` : Converts lat/long snake sightings into barangay-level records
- `aggregate_daily_counts.py` : Groups individual snake sightings into daily counts per barangay, preparing the data for time-based modeling.
using spatial join.
- `build_time_grid.py` : Expanded the aggregated sightings into a continuous daily time series for each barangay and species, ensuring all dates within the modeling window are explicitly represented.
- `poisson_redistribution` : Adjusts the daily snake sighting counts using a Poisson model to better represent how rare events naturally occur over time.
- `clean_env_count.py` : To clean the env data and counts to have the same order (by `adm4_psgc`), remove duplicates, and ensure `adm4_psgc` is `int` instead of `float
- `build_model_dataset.py` : combines both environmental data and snake sightings count
- `interpolation.py` : estimates missing env data values using nearby days

---
## Testing

In [22]:
import pandas as pd

original = pd.read_csv("/Users/sjao/Development/thesis-cobraSDM/data/daily_time_grid.csv")
poisson  = pd.read_csv("/Users/sjao/Development/thesis-cobraSDM/data/redistributed_counts.csv")

merged = original.merge(
    poisson,
    on=["barangay_psgc","species","date"],
    suffixes=("_orig","_pois")
)

changed_real_events = merged[
    (merged["count_orig"] > 0) &
    (merged["count_orig"] != merged["count_pois"])
]

print("Real events changed:", len(changed_real_events))


Real events changed: 0


In [23]:
new_nonzero = merged[
    (merged["count_orig"] == 0) &
    (merged["count_pois"] > 0)
]

print("Zero days converted to small counts:", len(new_nonzero))

Zero days converted to small counts: 983


In [24]:
print("Original mean:", original["count"].mean())
print("Poisson mean:", poisson["count"].mean())

print("Original max:", original["count"].max())
print("Poisson max:", poisson["count"].max())


Original mean: 0.00037536183678839877
Poisson mean: 0.0007210019623008478
Original max: 12
Poisson max: 12


In [2]:
import pandas as pd

df = pd.read_csv("/Users/sjao/Development/thesis-cobraSDM/data/redistributed_counts.csv")

# Ensure PSGC is string and no scientific notation
df["barangay_psgc"] = df["barangay_psgc"].astype(str)

unique_psgc = sorted(df["barangay_psgc"].unique())

print("Total unique barangays:", len(unique_psgc))

# Convert to JavaScript list format
js_list = 'var selectedPSGC = [\n'
js_list += ',\n'.join([f'  "{code}"' for code in unique_psgc])
js_list += '\n];'

print("\nCopy this into Earth Engine:\n")
print(js_list)



Total unique barangays: 669

Copy this into Earth Engine:

var selectedPSGC = [
  "1001304021.0",
  "1001305010.0",
  "1001310001.0",
  "1001310012.0",
  "1001312049.0",
  "1001313006.0",
  "1001314002.0",
  "1001314016.0",
  "1001315008.0",
  "1001315014.0",
  "1001317035.0",
  "1001318022.0",
  "1001322008.0",
  "1001801001.0",
  "1003522025.0",
  "1004203029.0",
  "1004206012.0",
  "1004210012.0",
  "1004210037.0",
  "1004212030.0",
  "1004214019.0",
  "1004215012.0",
  "1004306007.0",
  "1004307012.0",
  "1004308048.0",
  "1004310011.0",
  "1004312011.0",
  "1004316007.0",
  "1004319011.0",
  "102802046.0",
  "102804012.0",
  "102805018.0",
  "102809016.0",
  "102812061.0",
  "102812062.0",
  "102812084.0",
  "102817019.0",
  "102818010.0",
  "102820023.0",
  "102821020.0",
  "102823050.0",
  "102902015.0",
  "102905026.0",
  "102906039.0",
  "102908002.0",
  "102909005.0",
  "102910008.0",
  "102914030.0",
  "102916017.0",
  "102918006.0",
  "102924013.0",
  "102924032.0",
  "1029

---
### Current Best Working Code
#### Issues Found:
- No daily?
- Columns are weird

Current Output:
```Clean output:
FeatureCollection (1 element, 0 columns)
    type: FeatureCollection
    columns: Object (0 properties)
    features: List (1 element)
        0: Feature 000000000000000030ee
            type: Feature
            id: 000000000000000030ee
            geometry: null
            properties: Object (6 properties)
                LST_C: 25.561373059687906
                adm4_psgc: 802620013
                air_temperature: 298.7993094692041
                landcover: 1
                soil_moisture: 0.4456025619033543
                soil_temperature: 299.3547361280568

In [ ]:
```// -----------------------------
// SELECT BARANGAYS
// -----------------------------
var selectedPSGC = [
  102802046,
  102804012,
  102805018,
  102809016,
  102812061,
  102812062,
  102812084,
  102817019,
  102818010,
  102820023,
  102821020,
  102823050,
  102902015,
  102905026,
  102906039,
  102908002,
  102909005,
  102910008,
  102914030,
  102916017,
  102918006,
  102924013,
  102924032,
  102926005,
  102929001,
  102930002,
  102934003,
  102934040,
  103302010,
  103311039,
  103314007,
  103319010,
  105502011,
  105503021,
  105509013,
  105513026,
  105514011,
  105517019,
  105523016,
  105524062,
  105527070,
  105528014,
  105529011,
  105532016,
  105532078,
  105535001,
  105536020,
  105537023,
  105538028,
  105539015,
  105541006,
  105541024,
  105542011,
  105542014,
  105543005,
  105544047,
  105546027,
  105546032,
  105547002,
  201501007,
  201505020,
  201507008,
  201511008,
  201513052,
  201519017,
  201520019,
  201528032,
  201529013,
  201529018,
  201529024,
  203101002,
  203103027,
  203105004,
  203106018,
  203107021,
  203109007,
  203112066,
  203114067,
  203114100,
  203119007,
  203120013,
  203123014,
  203127008,
  203131024,
  203131029,
  203132019,
  203134012,
  203135025,
  203137031,
  205004011,
  205004012,
  205005017,
  205005021,
  205007019,
  205008018,
  205011001,
  205013013,
  205013014,
  205013021,
  205701005,
  205703001,
  205704038,
  300802016,
  300803005,
  300803013,
  300803020,
  300806009,
  300807001,
  300807008,
  300809016,
  300810001,
  300812004,
  301401014,
  301407021,
  301408010,
  301412020,
  301413022,
  301414009,
  301415016,
  301418003,
  301419026,
  301419033,
  301420009,
  301420062,
  301421037,
  301423016,
  301423017,
  301424004,
  301424006,
  304901021,
  304902021,
  304903046,
  304903077,
  304905030,
  304905053,
  304906018,
  304907004,
  304908013,
  304909019,
  304909020,
  304910017,
  304911007,
  304911060,
  304913016,
  304913020,
  304914004,
  304916003,
  304916021,
  304917029,
  304919020,
  304920012,
  304926019,
  304926024,
  304926035,
  304926039,
  304932004,
  305403018,
  305404003,
  305405029,
  305406023,
  305407004,
  305408021,
  305409005,
  305413013,
  305413024,
  305415031,
  305416001,
  305416010,
  305416027,
  305419004,
  306901013,
  306902010,
  306904007,
  306904008,
  306904021,
  306905035,
  306906029,
  306907019,
  306909020,
  306910016,
  306911013,
  306915021,
  306916046,
  306918004,
  306918005,
  306918013,
  307101022,
  307101032,
  307103015,
  307104011,
  307105003,
  307106006,
  307109014,
  307111017,
  307111019,
  307113024,
  307114004,
  307701009,
  307701010,
  307707006,
  307707025,
  307707031,
  330100003,
  330100021,
  331400006,
  401002015,
  401003009,
  401003037,
  401007006,
  401008027,
  401010017,
  401011017,
  401013003,
  401013019,
  401014038,
  401014042,
  401019007,
  401019022,
  401019042,
  401021032,
  401021033,
  401022022,
  401023029,
  401023034,
  401031043,
  401031049,
  401034008,
  402102016,
  402103004,
  402106008,
  402106056,
  402108004,
  402108012,
  402108014,
  402108021,
  402108027,
  402108028,
  402109028,
  402109053,
  402109059,
  402110023,
  402110034,
  402113008,
  402115023,
  402118020,
  402118032,
  402118040,
  402119008,
  402120002,
  402120006,
  402120016,
  402120024,
  402120027,
  402121001,
  402122011,
  403404006,
  403404014,
  403404015,
  403404017,
  403405026,
  403405052,
  403406003,
  403407015,
  403409003,
  403410018,
  403411014,
  403417034,
  403421002,
  403421004,
  403422004,
  403424007,
  403424008,
  403424031,
  403425011,
  403427020,
  403428012,
  403428020,
  403429013,
  403429022,
  403430010,
  405603005,
  405607015,
  405608001,
  405608019,
  405616009,
  405619041,
  405622008,
  405641014,
  405647032,
  405647057,
  405802008,
  405802009,
  405803010,
  405804007,
  405804013,
  405804020,
  405804024,
  405804035,
  405805014,
  405806022,
  405807015,
  405809006,
  405809008,
  405810001,
  405811006,
  405811013,
  405812012,
  405813002,
  431200008,
  431200011,
  500502047,
  500503048,
  500506041,
  500506049,
  500507007,
  500508015,
  500508023,
  500510014,
  500512026,
  500513026,
  500513029,
  500514013,
  500516011,
  500518028,
  501606035,
  501606046,
  501701016,
  501705023,
  501707009,
  501708001,
  501708028,
  501716025,
  501716030,
  501722024,
  501723033,
  501724012,
  501724013,
  501725016,
  501727017,
  501728001,
  501728020,
  501728026,
  501731014,
  501733023,
  501734040,
  501734043,
  501736015,
  502005008,
  502010020,
  504105027,
  504106008,
  504112011,
  504116015,
  506203009,
  506203033,
  506210019,
  506211027,
  506212035,
  506216018,
  600401011,
  600602038,
  600606033,
  600607043,
  600608021,
  600610014,
  600612025,
  600616062,
  603002054,
  603020030,
  603020033,
  603021034,
  603025030,
  603028071,
  603031019,
  603045016,
  604509008,
  604517012,
  604524013,
  604526002,
  607902026,
  701202005,
  701203015,
  701204014,
  701211016,
  701212005,
  701212018,
  701216001,
  701220030,
  701223019,
  701225030,
  701229018,
  701232004,
  701242010,
  701242013,
  701245026,
  701246017,
  701248012,
  702202004,
  702205035,
  702206011,
  702208025,
  702210038,
  702215016,
  702223029,
  702229015,
  702235020,
  702237016,
  702251028,
  702252050,
  706106019,
  730600068,
  730600074,
  730600082,
  802604039,
  802605024,
  802618005,
  802620013,
  803702051,
  803703010,
  803706025,
  803708066,
  803718031,
  803719028,
  803722021,
  803724004,
  803738012,
  803738087,
  803738115,
  803738116,
  803739023,
  803749012,
  804813019,
  804814032,
  806003023,
  806003032,
  806012020,
  806017039,
  806022044,
  806411016,
  806412006,
  806412010,
  806413003,
  807807016,
  831600047,
  831600121,
  907202004,
  907202012,
  907205037,
  907206002,
  907208010,
  907210022,
  907212016,
  907212020,
  907218025,
  907221010,
  907308025,
  907315020,
  907322021,
  907322056,
  907328013,
  907330023,
  908301019,
  908305023,
  908313042,
  908315021,
  931700005,
  931700013,
  931700060,
  931700104,
  990101040,
  990101049,
  1001304021,
  1001305010,
  1001310001,
  1001310012,
  1001312049,
  1001313006,
  1001314002,
  1001314016,
  1001315008,
  1001315014,
  1001317035,
  1001318022,
  1001322008,
  1001801001,
  1003522025,
  1004203029,
  1004206012,
  1004210012,
  1004210037,
  1004212030,
  1004214019,
  1004215012,
  1004306007,
  1004307012,
  1004308048,
  1004310011,
  1004312011,
  1004316007,
  1004319011,
  1030500067,
  1030500073,
  1030900016,
  1102301032,
  1102303007,
  1102305009,
  1102315025,
  1102317020,
  1102317041,
  1102319001,
  1102319005,
  1102319009,
  1102319010,
  1102319016,
  1102319023,
  1102319026,
  1102324013,
  1102401023,
  1102403014,
  1102403030,
  1102404013,
  1102410026,
  1102411012,
  1102503004,
  1102506010,
  1102508011,
  1102509005,
  1102510007,
  1108203006,
  1108205003,
  1108207013,
  1108208011,
  1108209028,
  1130700005,
  1130700014,
  1130700026,
  1130700030,
  1130700043,
  1130700044,
  1130700049,
  1130700056,
  1130700061,
  1130700066,
  1130700069,
  1130700104,
  1130700124,
  1130700135,
  1130700183,
  1204701006,
  1204703019,
  1204707029,
  1204708034,
  1204709037,
  1204714001,
  1206306003,
  1206306018,
  1206317014,
  1206502023,
  1206503022,
  1206503037,
  1206504005,
  1206506027,
  1206508020,
  1206509017,
  1208001010,
  1230800006,
  1230800007,
  1230800023,
  1230800033,
  1380800009,
  1381000003,
  1381300116,
  1381300124,
  1381300141,
  1381500025,
  1400101028,
  1400113002,
  1400119016,
  1401101002,
  1401101005,
  1401103009,
  1401104001,
  1401104006,
  1401104009,
  1401106001,
  1401106004,
  1401106005,
  1401107013,
  1401110008,
  1401110014,
  1401111009,
  1401111012,
  1401112011,
  1401113002,
  1401113007,
  1401114006,
  1402701004,
  1402701007,
  1402702011,
  1402707017,
  1403211007,
  1403213008,
  1403215014,
  1404401006,
  1404402020,
  1404404016,
  1404410017,
  1408101007,
  1408102003,
  1408104018,
  1430300002,
  1430300025,
  1430300050,
  1430300052,
  1430300119,
  1430300149,
  1600201003,
  1600201025,
  1600203019,
  1600204012,
  1600206006,
  1600206011,
  1600208009,
  1600301016,
  1600306018,
  1600308019,
  1600311006,
  1606706009,
  1606714006,
  1606724026,
  1606724066,
  1606724068,
  1606725007,
  1606801001,
  1606803009,
  1606803012,
  1606813007,
  1606814006,
  1606819005,
  1608504001,
  1608504015,
  1630400027,
  1630400051,
  1630400055,
  1630400089,
  1704001035,
  1704001061,
  1705101004,
  1705111008,
  1705111011,
  1705215004,
  1705304024,
  1705306018,
  1705309017,
  1705312005,
  1705319007,
  1731500067,
  1903602002,
  1903615072,
  1903617083,
  1903619038,
  1903624077,
  1903629014,
  1903630034,
  1903630042,
  1908705013,
  1908705014,
  1908822004,
  1999902006
];

// =============================
// 2. DATE RANGE
// =============================
var startDate = '2022-01-01';
var endDate   = '2026-01-31';

var barangays = ee.FeatureCollection(
  'projects/cobra-sdm-487706/assets/PH_Adm4_BgySubMuns'
).filter(ee.Filter.inList('adm4_psgc', selectedPSGC));

print('Selected barangays:', barangays.size());

// =============================
// 3. CLIP REGION FIRST
// =============================
var region = barangays.geometry();

// =============================
// 4. ERA5 (NO REPROJECT)
// =============================
var era5 = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
  .filterDate(startDate, endDate)
  .select([
    'temperature_2m',
    'soil_temperature_level_1',
    'volumetric_soil_water_layer_1'
  ])
  .mean()
  .clip(region);

// =============================
// 5. MODIS LST (NO REPROJECT)
// =============================
var lst = ee.ImageCollection('MODIS/061/MOD11A1')
  .filterDate(startDate, endDate)
  .select('LST_Day_1km')
  .mean()
  .multiply(0.02)
  .subtract(273.15)
  .rename('LST_C')
  .clip(region);

// =============================
// 6. LAND COVER (Dynamic World)
// =============================
var landcover = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
  .filterDate(startDate, endDate)
  .select('label')
  .mode()
  .rename('landcover')
  .clip(region);

// =============================
// 7. MERGE BANDS
// =============================
var merged = era5
  .addBands(lst)
  .addBands(landcover);

print('Merged bands:', merged.bandNames());

// =============================
// 8. REDUCE PER BARANGAY
// =============================
var reduced = merged.reduceRegions({
  collection: barangays,
  reducer: ee.Reducer.mean(),
  scale: 1000,
  tileScale: 4,
  maxPixelsPerRegion: 1e13
});

// Keep only required fields
var clean = reduced.map(function(feature) {
  return ee.Feature(null, {
    adm4_psgc: feature.get('adm4_psgc'),
    LST_C: feature.get('LST_C'),
    soil_temperature: feature.get('soil_temperature_level_1'),
    air_temperature: feature.get('temperature_2m'),
    soil_moisture: feature.get('volumetric_soil_water_layer_1'),
    landcover: feature.get('landcover')
  });
});

print("Clean output:", clean.limit(1));```

In [2]:
import pandas as pd

df = pd.read_csv("/Users/sjao/Development/thesis-cobraSDM/data/model_dataset.csv")

print("Unique species:", df["species"].unique())
print("Date range:", df["date"].min(), "to", df["date"].max())

print("Rows per barangay-date:")
print(
    df.groupby(["barangay_psgc", "date"])
      .size()
      .describe()
)

Unique species: ['Naja philippinensis' 'Naja samarensis' 'Ophiophagus hannah']
Date range: 2022-01-01 to 2026-01-31
Rows per barangay-date:
count    998148.0
mean          3.0
std           0.0
min           3.0
25%           3.0
50%           3.0
75%           3.0
max           3.0
dtype: float64
